In this script, the previously ditch/stream classified vector line geometries are rasterized and compared (summed) to a raster mask

In [ ]:
import numpy as np
import geopandas as gpd
import pandas as pd
import dask_geopandas as dask_gpd

# Ditch/Stream split
The dataset was first split into ditch and stream like features with threshold value 0.5 predicted_class. In this combined raster, ditch like features are emphasized. If both rasters have data (not equal to 0), “ditch_50m@1” raster values are used. 

Other option is to use a custom filter for the split. In this split, additional parameters were used in addition to the predicted_class as it seemed to overestimate the number of natural streams. Use the predicted_class if identification of streams is of high importance as this custom filter might see some natural streams as ditches. Lapland is an exception as there is a minimum amount of ditched areas.

In [ ]:
#using predicted_class

filename = r"Path\ClassifiedStreams.parquet"
ddf = dask_gpd.read_parquet(filename, npartitions=2)
df = ddf.compute()
print(len(df))

ditch = df.predicted_class_mean <= 0.5
stream = df.predicted_class_mean > 0.5

df[ditch].to_parquet(r"Path\ditch.parquet")
print(len(df[ditch]))
df[stream].to_parquet(r"Path\stream.parquet")
print(len(df[stream]))

18775792
16518532
2257260


In [ ]:
#using custom filter

filename = r"Path\ClassifiedStreams.parquet"
ddf = dask_gpd.read_parquet(filename, npartitions=2)
df = ddf.compute()
print(len(df))

stream = df[
    (df['num_vertices'] > 6) &
    (df['sinuosity'] > 1.02) &
    (df['mean_distance_seq'] < 10) &
    ((df['StreamProbRF'] > 0.6) | (df['StreamProbNN'] > 0.6))
]

stream.to_parquet(r"Path\stream.parquet")
print(len(stream))
      
ditch = df[
    ~(
        (df['num_vertices'] > 6) &
        (df['sinuosity'] > 1.02) &
        (df['mean_distance_seq'] < 10) &
        ((df['StreamProbRF'] > 0.6) | (df['StreamProbNN'] > 0.6))
    )
]

ditch.to_parquet(r"Path\ditch.parquet")
print(len(ditch))

18775792
344469
18431323


# Buffer

In [ ]:
#if you have a big dataset not fitting into memory, run in parts
#run for both ditch and stream and comment the other option out
filename = r"Path\ditch.parquet"
filename = r"Path\stream.parquet"
ddf = dask_gpd.read_parquet(filename, npartitions=8)
ddf = ddf.repartition(npartitions=8)

npartition=8
for n in range(npartition):
    # Get the 0th partition
    partition = ddf.get_partition(n)
    # Perform the buffer operation
    buffer_distance = 50  # Buffer distance
    partition['geometry'] = partition['geometry'].buffer(buffer_distance)
    # Write the result to disk to free up memory
    df = partition.compute()
    out = rf"Path\ditch_buff50m.parquet{n}.parquet"
    out = rf"Path\stream_buff50m.parquet{n}.parquet"
    print(n, len(df))
    df.to_parquet(out)

#spectate layers in qgis and merge them there
do for both ditch and stream
processing.run("native:mergevectorlayers", {'LAYERS':['Path/stream_buff50m.parquet0.parquet|layername=stream_buff50m.parquet0','Path/stream_buff50m.parquet1.parquet|layername=stream_buff50m.parquet1','Path/stream_buff50m.parquet2.parquet|layername=stream_buff50m.parquet2','Path/stream_buff50m.parquet3.parquet|layername=stream_buff50m.parquet3','Path/stream_buff50m.parquet4.parquet|layername=stream_buff50m.parquet4','Path/stream_buff50m.parquet5.parquet|layername=stream_buff50m.parquet5','Path/stream_buff50m.parquet6.parquet|layername=stream_buff50m.parquet6','Path/stream_buff50m.parquet7.parquet|layername=stream_buff50m.parquet7'],'CRS':QgsCoordinateReferenceSystem('EPSG:3067'),'OUTPUT':'Path/stream_buff50m_all.parquet'})

# Rasterize
-with mask properties (resolution, extent, crs...)

Run for both ditch and stream

processing.run("gdal:rasterize", {'INPUT':'YourFile','FIELD':'predicted_class_mean','BURN':0,'USE_Z':False,'UNITS':0,'WIDTH':69600,'HEIGHT':118800,'EXTENT':'44000.000000000,740000.000000000,6594000.000000000,7782000.000000000 [EPSG:3067]','NODATA':0,'OPTIONS':'','DATA_TYPE':5,'INIT':None,'INVERT':False,'EXTRA':'','OUTPUT':'Path/50mbuff.tif'})

# Assing minimum value or round up (optional)

With float32 in some cases it seemed that e.g. a raster sum 1+3.29e-9 or similar might end up in a sum of 1.00 as the latter is summed to 0.00 due to lacking precision. dtype='float64' might work in some cases

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

file = r"Path\ditch_50mbuff_compressed.tif"
out = r"Path\ditch_50mbuff_MinAdjusted.tif"

# Open the raster file and convert it into a DataArray
ditch = rioxarray.open_rasterio(file, chunks={'band': 1, 'x': 1024, 'y': 1024})

# Define the minimum value
min_value = 0.001

# Apply the minimum value
adjusted_ditch = ditch.where(ditch < min_value, min_value)

# Save the adjusted raster as a new file
adjusted_ditch.rio.to_raster(out)

# Combine rasters to emphasize ditch effect

If both rasters have data (not equal to 0), “ditch_50m@1” raster values are used.
If only the “stream_50m@1” raster has data, “stream_50m@1” raster values are used.
If only the “ditch_50m@1” raster has data, “ditch_50m@1” raster values are used.

The idea is that if there are ditches close to natural streams, their impact "overwrites"
the natural impact

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

out = r'Path\CombinedStreamRaster.tif'

# Read the data in chunks
ditch = rioxarray.open_rasterio(r'Path\ditch_50mbuff.tif', chunks={'band': 1, 'x': 1024, 'y': 1024})
stream = rioxarray.open_rasterio(r'Path\stream_50mbuff.tif', chunks={'band': 1, 'x': 1024, 'y': 1024})

ditch_data = ditch[0]
print('ditch read')
stream_data = stream[0]
print('stream read')

# Create a new array to hold the result
result = da.zeros_like(ditch_data)
print('new array done')

# Apply the conditions
result = da.where((ditch_data != 0) & (stream_data != 0), ditch_data, result)
result = da.where((ditch_data == 0) & (stream_data != 0), stream_data, result)
result = da.where((ditch_data != 0) & (stream_data == 0), ditch_data, result)
print('conditions set')

# Save the result to a new raster file
result_xarray = xr.DataArray(result, dims=("y", "x"), coords={"y": ditch.y, "x": ditch.x})
result_xarray.rio.write_crs(ditch.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='float32')


# Add 50m buffered peat production areas, fields and roads as ditches

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

FieldRoadPeatprod = r"Path\Masks\TiePeltoTurvetuotantoBuffer50m.tif"
streams = r"Path\all_buff50m.tif"
out = r'Path\CombinedStreamRaster.tif'

# Read the data in chunks
streams = rioxarray.open_rasterio(streams, chunks={'band': 1, 'x': 2048, 'y': 2048})
FieldRoadPeatprod = rioxarray.open_rasterio(FieldRoadPeatprod, chunks={'band': 1, 'x': 2048, 'y': 2048})

streams_data = streams[0]
print('streams read')
FieldRoadPeatprod_data = FieldRoadPeatprod[0]
print('FieldRoadPeatprod read')

# Apply the conditions
streams_data = da.where(FieldRoadPeatprod_data != 0, 1, streams_data) #if fieldroadpeatland, then fieldroadpeatland else stream
print('conditions set')

# Save the result to a new raster file
result_xarray = xr.DataArray(streams_data, dims=("y", "x"), coords={"y": streams.y, "x": streams.x})
result_xarray.rio.write_crs(streams.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='int8', compress='LZW', sparse_ok=True)

ditch read
stream read
new array done
conditions set


# Compare with mask

-if categorical case, uint8 addition (e.g. 1+4) is enough without astype(np.float32)

-if probabilistic case and the mask has uint8 values, they need to be converted to float first. 
Here mask has values 1 and 2, and the stream raster has float32 probablility values 0 - 1.
by summing the values, the values in mask raster that do not overlap, remain unchanged (can be filtered with values 1.00 and 2.00) and
-range 1.01 - 1.99 are the mask class 1 pixels that are on the vector line (ditch/stream) impact area (50 m buffer)
-range 2.01 - 2.99 are the mask class 2 pixels that are on the vector line (ditch/stream) impact area (50 m buffer)

both rasters are of same properties (resolution, extent, crs), if in your case not, resample first

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

Mask = r"Path\Mask.tif"
Streams = r"Path\CombinedStreamraster.tif"
out = r"Path\MaskWithStreamvalues.tif"

# Open the raster files with chunking
mask = rioxarray.open_rasterio(Mask, chunks={'band': 1, 'x': 1024, 'y': 1024})
stream = rioxarray.open_rasterio(Streams, chunks={'band': 1, 'x': 1024, 'y': 1024})

# Add the values of stream to ditch
result = (mask[0] * 10) + stream[0]
# reclassify, mineral soil and peatland with natural streams to mineral soil and peatland
result = da.where(result == 22, 20, result)
result = da.where(result == 12, 10, result)

# Save the result to a new raster file
result_xarray = xr.DataArray(result, dims=("y", "x"), coords={"y": mask.y, "x": mask.x})
result_xarray.rio.write_crs(mask.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='uint8', compress='LZW', sparse_ok=True) #np.float32 if probability

print('done')

# Add additional classes that overwrite the base layer
22 = Peat field (that overwrites the previous classes), 23 = Peat production areas (that overwrites the previous classes)

In [ ]:
#Peat fields

import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

MaskWithStreams = r"path\MaskWithStreamvalues.tif"
turvepelto = r"path\turvepelto.tif"
out = r"path\MaskWithStreamvalues_TurvPelt.tif"

# Read the data in chunks
MaskWithStreams = rioxarray.open_rasterio(MaskWithStreams, chunks={'band': 1, 'x': 2048, 'y': 2048})
turvepelto = rioxarray.open_rasterio(turvepelto, chunks={'band': 1, 'x': 2048, 'y': 2048})
turvepelto = turvepelto.rio.reproject_match(MaskWithStreams) #second raster in a different shape so its converted to match

MaskWithStreams_data = MaskWithStreams[0]
print('MaskWithStreams read')
turvepelto_data = turvepelto[0]
print('turvepelto read')

# Apply the conditions
MaskWithStreams_data = da.where(turvepelto_data != 0, 22, MaskWithStreams_data) #if turvepelto, then turvepelto else mask
print('conditions set')

# Save the result to a new raster file
result_xarray = xr.DataArray(MaskWithStreams_data, dims=("y", "x"), coords={"y": MaskWithStreams.y, "x": MaskWithStreams.x})
result_xarray.rio.write_crs(MaskWithStreams.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='int8', compress='LZW', sparse_ok=True)

In [ ]:
# Peat production areas

import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

MaskWithStreams = r"path\MaskWithStreamvalues_TurvPelt.tif"
turvetuotantoalue = r"path\Turvetuotanto.tif"
out = r"path\MaskWithStreamvalues_TurvPelt_Turvetuot.tif"

# Read the data in chunks
MaskWithStreams = rioxarray.open_rasterio(MaskWithStreams, chunks={'band': 1, 'x': 2048, 'y': 2048})
turvetuotantoalue = rioxarray.open_rasterio(turvetuotantoalue, chunks={'band': 1, 'x': 2048, 'y': 2048})

MaskWithStreams_data = MaskWithStreams[0]
print('MaskWithStreams read')
turvetuotantoalue_data = turvetuotantoalue[0]
print('turvetuotantoalue read')

# Apply the conditions
MaskWithStreams_data = da.where(turvetuotantoalue_data != 0, 23, MaskWithStreams_data) #if turvetuotantoalue, then turvetuotantoalue else mask
print('conditions set')

# Save the result to a new raster file
result_xarray = xr.DataArray(MaskWithStreams_data, dims=("y", "x"), coords={"y": MaskWithStreams.y, "x": MaskWithStreams.x})
result_xarray.rio.write_crs(MaskWithStreams.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='int8', compress='LZW', sparse_ok=True)

# Compress end product for storage
-if not done previously
-by default this script compresses the output
result_xarray.rio.to_raster(out, dtype='int8', compress='LZW', sparse_ok=True)

In [ ]:
#Done in qgis

Algorithm 'Translate (convert format)' starting…
Input parameters:
{ 'COPY_SUBDATASETS' : False, 'DATA_TYPE' : 0, 'EXTRA' : '', 'INPUT' : 'Path.tif', 'NODATA' : None, 'OPTIONS' : 'COMPRESS=LZW|TILED=YES|SPARSE_OK=YES', 'OUTPUT' : 'Path/_compressed.tif', 'TARGET_CRS' : None }

GDAL command:
gdal_translate -of GTiff -co COMPRESS=LZW -co TILED=YES -co SPARSE_OK=YES path path
GDAL command output:
Input file size is 69600, 118800 